# CrewAI Custom Tools: Practical Applications

This notebook demonstrates how to create and use custom tools in CrewAI using function-based approaches. We'll explore various real-world applications where custom tools enhance agent capabilities.

## 1. Setup and Installation

In [ ]:
!pip install crewai crewai-tools langchain-openai python-dotenv pandas requests matplotlib seaborn

In [ ]:
import os
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from typing import Dict, List, Any

from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()


# Set up API keys (you'll need to set these in your environment)
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["SERPER_API_KEY"] = "your-serper-api-key"  # For web search
os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_ROUTER_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ['LITELLM_LOG'] = 'DEBUG' 
os.environ['OPENAI_API_BASE'] = 'https://openrouter.ai/api/v1'
os.environ['OPENAI_BASE_URL'] = 'https://openrouter.ai/api/v1'


# Initialize the language model
llm = LLM(
        model='openai/gpt-4o',
        api_key=os.getenv('OPEN_ROUTER_KEY'),
        base_url="https://openrouter.ai/api/v1"
    )

## 2. Custom Tool Basics

In CrewAI, you can create custom tools using the `@tool` decorator. This is simpler than creating class-based tools.

**Important Note:** When you use the `@tool` decorator, the function becomes a tool object that is not directly callable. The tool is meant to be used by agents, not called directly in your code.

Here are three methods to handle this:

In [ ]:
# Method 1: Define function first, then create tool
def calculate_compound_interest_func(principal: float, rate: float, time: int, n: int = 12) -> str:
    """
    Calculate compound interest.
    
    Args:
        principal: Initial amount
        rate: Annual interest rate (as decimal, e.g., 0.05 for 5%)
        time: Time period in years
        n: Number of times interest compounds per year
    """
    amount = principal * (1 + rate/n) ** (n * time)
    interest = amount - principal
    
    return f"""
    Principal: ${principal:,.2f}
    Rate: {rate*100}%
    Time: {time} years
    Final Amount: ${amount:,.2f}
    Interest Earned: ${interest:,.2f}
    """

# Create the tool
calculate_compound_interest = tool("Calculate Compound Interest")(calculate_compound_interest_func)

# Test the function directly
print("Direct function call:")
print(calculate_compound_interest_func(10000, 0.05, 10))

# The tool object itself is not callable, but can be used by agents
print("\nTool object:", type(calculate_compound_interest))

In [ ]:
# Method 2: Use tool decorator but access the function for testing
@tool("Simple Calculator")
def simple_calculator(a: float, b: float, operation: str) -> str:
    """
    Perform simple arithmetic operations.
    
    Args:
        a: First number
        b: Second number
        operation: Operation to perform (add, subtract, multiply, divide)
    """
    operations = {
        'add': a + b,
        'subtract': a - b,
        'multiply': a * b,
        'divide': a / b if b != 0 else 'Error: Division by zero'
    }
    
    result = operations.get(operation, 'Invalid operation')
    return f"{a} {operation} {b} = {result}"

# For testing purposes, you can access the underlying function
# Note: This method may vary depending on CrewAI version
try:
    # Try to access the function directly if stored as an attribute
    if hasattr(simple_calculator, 'func'):
        print(simple_calculator.func(10, 5, 'add'))
    elif hasattr(simple_calculator, '_run'):
        print(simple_calculator._run(10, 5, 'add'))
    else:
        print("Tool created successfully but can't access function directly")
        print("This tool will work when used by an agent")
except Exception as e:
    print(f"Note: {e}")
    print("The tool is ready to be used by agents")

### Alternative Approach: Separate Functions and Tools

A clean pattern is to maintain your functions separately from the tool objects. This makes testing easier and keeps your code more modular.

In [ ]:
# Define all your functions in one place
def text_analyzer(text: str) -> str:
    """Analyze text and return statistics."""
    words = text.split()
    chars = len(text)
    sentences = text.count('.') + text.count('!') + text.count('?')
    
    return f"""
    Text Analysis:
    - Characters: {chars}
    - Words: {len(words)}
    - Sentences: {sentences}
    - Avg words per sentence: {len(words)/max(sentences, 1):.1f}
    """

def word_counter(text: str, target_word: str) -> str:
    """Count occurrences of a specific word."""
    count = text.lower().count(target_word.lower())
    return f"The word '{target_word}' appears {count} time(s) in the text."

# Test your functions normally
test_text = "This is a test. This is only a test!"
print(text_analyzer(test_text))
print(word_counter(test_text, "test"))

# When ready to use with agents, create the tools
text_analyzer_tool = tool("Text Analyzer")(text_analyzer)
word_counter_tool = tool("Word Counter")(word_counter)

print("\nTools created successfully!")

## 3. Application 1: Data Analysis Tools

Custom tools for analyzing datasets, calculating statistics, and generating insights.

In [ ]:
# Define the functions first, then create tools
def analyze_csv_data_func(file_path: str, column: str = None) -> str:
    """
    Analyze CSV data and provide statistical insights.
    
    Args:
        file_path: Path to CSV file
        column: Specific column to analyze (optional)
    """
    try:
        df = pd.read_csv(file_path)
        
        analysis = f"Dataset Overview:\n"
        analysis += f"- Shape: {df.shape[0]} rows, {df.shape[1]} columns\n"
        analysis += f"- Columns: {', '.join(df.columns)}\n\n"
        
        if column and column in df.columns:
            if pd.api.types.is_numeric_dtype(df[column]):
                analysis += f"Statistics for '{column}':\n"
                analysis += f"- Mean: {df[column].mean():.2f}\n"
                analysis += f"- Median: {df[column].median():.2f}\n"
                analysis += f"- Std Dev: {df[column].std():.2f}\n"
                analysis += f"- Min: {df[column].min()}\n"
                analysis += f"- Max: {df[column].max()}\n"
            else:
                analysis += f"Value counts for '{column}':\n"
                for val, count in df[column].value_counts().head(5).items():
                    analysis += f"- {val}: {count}\n"
        else:
            # General statistics for numeric columns
            numeric_cols = df.select_dtypes(include=['number']).columns
            if len(numeric_cols) > 0:
                analysis += "Numeric Column Summary:\n"
                for col in numeric_cols[:3]:  # First 3 numeric columns
                    analysis += f"\n{col}:"
                    analysis += f" Mean={df[col].mean():.2f}, Std={df[col].std():.2f}"
        
        return analysis
    except Exception as e:
        return f"Error analyzing data: {str(e)}"

def create_data_visualization_func(data: dict, chart_type: str = "bar", title: str = "Data Visualization") -> str:
    """
    Create visualizations from data.
    
    Args:
        data: Dictionary with labels as keys and values as values
        chart_type: Type of chart (bar, line, pie)
        title: Chart title
    """
    try:
        plt.figure(figsize=(10, 6))
        
        if chart_type == "bar":
            plt.bar(data.keys(), data.values())
            plt.xlabel('Categories')
            plt.ylabel('Values')
        elif chart_type == "line":
            plt.plot(list(data.keys()), list(data.values()), marker='o')
            plt.xlabel('X-axis')
            plt.ylabel('Y-axis')
        elif chart_type == "pie":
            plt.pie(data.values(), labels=data.keys(), autopct='%1.1f%%')
        
        plt.title(title)
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        # Save the plot
        filename = f"visualization_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
        plt.savefig(filename)
        plt.close()
        
        return f"Visualization saved as {filename}. Data points: {len(data)}"
    except Exception as e:
        return f"Error creating visualization: {str(e)}"

# Create tools from functions
analyze_csv_data = tool("CSV Data Analyzer")(analyze_csv_data_func)
create_data_visualization = tool("Data Visualization Generator")(create_data_visualization_func)

# Test the functions
print("Testing visualization function:")
test_data = {"Category A": 100, "Category B": 150, "Category C": 75}
print(create_data_visualization_func(test_data, "bar", "Test Chart"))

In [ ]:
# Create a data analyst agent with custom tools
data_analyst = Agent(
    role="Data Analyst",
    goal="Analyze data and provide insights using custom analytical tools",
    backstory="""
    You are an experienced data analyst who excels at finding patterns
    and insights in data. You use various analytical tools to process
    and visualize data effectively.
    """,
    tools=[analyze_csv_data, create_data_visualization],
    llm=llm,
    verbose=True
)

## 4. Application 2: API Integration Tools

Custom tools for integrating with external APIs and services.

In [ ]:
# Define API integration functions
def get_weather_data_func(city: str) -> str:
    """
    Get current weather data for a city.
    
    Args:
        city: Name of the city
    """
    # This is a mock implementation - replace with actual API
    mock_data = {
        "London": {"temp": 15, "condition": "Cloudy", "humidity": 75},
        "New York": {"temp": 22, "condition": "Sunny", "humidity": 60},
        "Tokyo": {"temp": 18, "condition": "Rainy", "humidity": 85}
    }
    
    if city in mock_data:
        data = mock_data[city]
        return f"""
        Weather in {city}:
        - Temperature: {data['temp']}°C
        - Condition: {data['condition']}
        - Humidity: {data['humidity']}%
        """
    else:
        return f"Weather data not available for {city}"

def get_stock_price_func(symbol: str) -> str:
    """
    Fetch current stock price and related information.
    
    Args:
        symbol: Stock ticker symbol
    """
    # Mock implementation - replace with actual API like yfinance
    mock_prices = {
        "AAPL": {"price": 178.25, "change": 2.15, "volume": "52M"},
        "GOOGL": {"price": 142.80, "change": -1.20, "volume": "28M"},
        "MSFT": {"price": 378.90, "change": 3.45, "volume": "41M"}
    }
    
    if symbol.upper() in mock_prices:
        data = mock_prices[symbol.upper()]
        change_pct = (data['change'] / data['price']) * 100
        return f"""
        Stock: {symbol.upper()}
        Price: ${data['price']}
        Change: ${data['change']} ({change_pct:+.2f}%)
        Volume: {data['volume']}
        """
    else:
        return f"Stock data not available for {symbol}"

def convert_currency_func(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one to another.
    
    Args:
        amount: Amount to convert
        from_currency: Source currency code
        to_currency: Target currency code
    """
    # Mock exchange rates - replace with actual API
    rates = {
        "USD": 1.0,
        "EUR": 0.85,
        "GBP": 0.73,
        "JPY": 110.0,
        "CAD": 1.25
    }
    
    if from_currency in rates and to_currency in rates:
        # Convert to USD first, then to target currency
        usd_amount = amount / rates[from_currency]
        converted = usd_amount * rates[to_currency]
        
        return f"""
        Currency Conversion:
        {amount:,.2f} {from_currency} = {converted:,.2f} {to_currency}
        Exchange rate: 1 {from_currency} = {rates[to_currency]/rates[from_currency]:.4f} {to_currency}
        """
    else:
        return f"Currency conversion not available for {from_currency} to {to_currency}"

# Create tools
get_weather_data = tool("Weather API Tool")(get_weather_data_func)
get_stock_price = tool("Stock Price Fetcher")(get_stock_price_func)
convert_currency = tool("Currency Converter")(convert_currency_func)

# Test the functions
print("Testing API functions:")
print(get_weather_data_func("London"))
print(get_stock_price_func("AAPL"))
print(convert_currency_func(100, "USD", "EUR"))

## 5. Application 3: File Processing Tools

Tools for handling various file operations and transformations.

In [ ]:
# File processing functions
def process_text_file_func(file_path: str, operation: str = "stats") -> str:
    """
    Process text files with various operations.
    
    Args:
        file_path: Path to text file
        operation: Operation to perform (stats, summary, keywords)
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        if operation == "stats":
            words = content.split()
            lines = content.split('\n')
            return f"""
            File Statistics:
            - Characters: {len(content)}
            - Words: {len(words)}
            - Lines: {len(lines)}
            - Average words per line: {len(words)/len(lines):.1f}
            """
        
        elif operation == "summary":
            # Simple summary: first 3 lines
            lines = content.split('\n')[:3]
            return f"First 3 lines:\n" + "\n".join(lines)
        
        elif operation == "keywords":
            # Simple keyword extraction: most common words
            words = content.lower().split()
            word_freq = {}
            for word in words:
                if len(word) > 4:  # Only words longer than 4 chars
                    word_freq[word] = word_freq.get(word, 0) + 1
            
            top_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:5]
            keywords = "\n".join([f"- {word}: {count}" for word, count in top_words])
            return f"Top Keywords:\n{keywords}"
            
    except Exception as e:
        return f"Error processing file: {str(e)}"

def transform_json_data_func(json_str: str, transformation: str = "flatten") -> str:
    """
    Transform JSON data in various ways.
    
    Args:
        json_str: JSON string to transform
        transformation: Type of transformation (flatten, extract_keys, format)
    """
    try:
        data = json.loads(json_str)
        
        if transformation == "flatten":
            # Simple flattening
            def flatten_dict(d, parent_key='', sep='.'):
                items = []
                for k, v in d.items():
                    new_key = f"{parent_key}{sep}{k}" if parent_key else k
                    if isinstance(v, dict):
                        items.extend(flatten_dict(v, new_key, sep=sep).items())
                    else:
                        items.append((new_key, v))
                return dict(items)
            
            if isinstance(data, dict):
                flattened = flatten_dict(data)
                return f"Flattened JSON:\n{json.dumps(flattened, indent=2)}"
        
        elif transformation == "extract_keys":
            def get_all_keys(d, keys=set()):
                if isinstance(d, dict):
                    for k, v in d.items():
                        keys.add(k)
                        if isinstance(v, dict):
                            get_all_keys(v, keys)
                        elif isinstance(v, list):
                            for item in v:
                                if isinstance(item, dict):
                                    get_all_keys(item, keys)
                return keys
            
            all_keys = get_all_keys(data)
            return f"All keys in JSON:\n" + "\n".join(f"- {k}" for k in sorted(all_keys))
        
        elif transformation == "format":
            return f"Formatted JSON:\n{json.dumps(data, indent=2, sort_keys=True)}"
            
    except json.JSONDecodeError as e:
        return f"Error parsing JSON: {str(e)}"
    except Exception as e:
        return f"Error transforming data: {str(e)}"

# Create tools
process_text_file = tool("Text File Processor")(process_text_file_func)
transform_json_data = tool("JSON Data Transformer")(transform_json_data_func)

# Test JSON transformation
test_json = '{"name": "John", "age": 30, "address": {"city": "NYC", "zip": "10001"}}'
print("Testing JSON transformation:")
print(transform_json_data_func(test_json, "extract_keys"))

## 6. Application 4: Business Logic Tools

Custom tools implementing specific business logic and calculations.

In [ ]:
# Define the business logic functions first
def calculate_roi_func(initial_investment: float, final_value: float, time_period: float = 1) -> str:
    """Calculate Return on Investment (ROI) with various metrics."""
    roi = ((final_value - initial_investment) / initial_investment) * 100
    total_return = final_value - initial_investment
    annualized_roi = (((final_value / initial_investment) ** (1/time_period)) - 1) * 100
    
    return f"""
    ROI Analysis:
    - Initial Investment: ${initial_investment:,.2f}
    - Final Value: ${final_value:,.2f}
    - Total Return: ${total_return:,.2f}
    - ROI: {roi:.2f}%
    - Annualized ROI: {annualized_roi:.2f}%
    - Time Period: {time_period} years
    """

def calculate_clv_func(avg_purchase_value: float, purchase_frequency: float, 
                      customer_lifespan: float, profit_margin: float = 0.2) -> str:
    """Calculate Customer Lifetime Value (CLV)."""
    annual_revenue = avg_purchase_value * purchase_frequency
    clv = annual_revenue * customer_lifespan
    profit_clv = clv * profit_margin
    
    return f"""
    Customer Lifetime Value Analysis:
    - Average Purchase Value: ${avg_purchase_value:,.2f}
    - Purchase Frequency: {purchase_frequency} per year
    - Customer Lifespan: {customer_lifespan} years
    - Annual Revenue per Customer: ${annual_revenue:,.2f}
    - Total CLV: ${clv:,.2f}
    - Profit CLV (at {profit_margin*100}% margin): ${profit_clv:,.2f}
    """

def calculate_break_even_func(fixed_costs: float, variable_cost_per_unit: float, 
                             price_per_unit: float) -> str:
    """Calculate break-even point for a business."""
    if price_per_unit <= variable_cost_per_unit:
        return "Error: Price must be greater than variable cost per unit"
    
    contribution_margin = price_per_unit - variable_cost_per_unit
    break_even_units = fixed_costs / contribution_margin
    break_even_revenue = break_even_units * price_per_unit
    margin_ratio = contribution_margin / price_per_unit
    
    return f"""
    Break-Even Analysis:
    - Fixed Costs: ${fixed_costs:,.2f}
    - Variable Cost per Unit: ${variable_cost_per_unit:.2f}
    - Price per Unit: ${price_per_unit:.2f}
    - Contribution Margin: ${contribution_margin:.2f}
    - Break-Even Units: {break_even_units:,.0f} units
    - Break-Even Revenue: ${break_even_revenue:,.2f}
    - Contribution Margin Ratio: {margin_ratio*100:.1f}%
    """

# Create tools from functions
calculate_roi = tool("ROI Calculator")(calculate_roi_func)
calculate_clv = tool("Customer Lifetime Value")(calculate_clv_func)
calculate_break_even = tool("Break-Even Analysis")(calculate_break_even_func)

# Test the functions
print("Testing ROI calculation:")
print(calculate_roi_func(100000, 150000, 2))

## 7. Application 5: Automation and Workflow Tools

Tools for automating repetitive tasks and managing workflows.

In [ ]:
# Automation and workflow functions
def generate_email_template_func(template_type: str, recipient_name: str, 
                                custom_data: dict = None) -> str:
    """
    Generate email templates for various purposes.
    
    Args:
        template_type: Type of email (welcome, follow_up, invoice)
        recipient_name: Name of the recipient
        custom_data: Additional data for customization
    """
    if custom_data is None:
        custom_data = {}
        
    templates = {
        "welcome": f"""
Subject: Welcome to Our Service, {recipient_name}!

Dear {recipient_name},

We're thrilled to have you join our community! Your account has been successfully created.

Here's what you can do next:
- Complete your profile
- Explore our features
- Connect with our support team if you need help

Best regards,
The Team
""",
        "follow_up": f"""
Subject: Following Up on Our Recent Discussion

Hi {recipient_name},

I wanted to follow up on our conversation from earlier this week.

{custom_data.get('context', 'We discussed several important points.')} 

Please let me know if you have any questions or if there's anything else I can help you with.

Looking forward to hearing from you.

Best regards
""",
        "invoice": f"""
Subject: Invoice #{custom_data.get('invoice_number', '00001')} - Payment Due

Dear {recipient_name},

Please find below the details of your invoice:

Invoice Number: {custom_data.get('invoice_number', '00001')}
Amount Due: ${custom_data.get('amount', '0.00')}
Due Date: {custom_data.get('due_date', 'Upon receipt')}

Payment can be made via bank transfer or credit card.

Thank you for your business.

Accounts Department
"""
    }
    
    if template_type in templates:
        return templates[template_type]
    else:
        return f"Template type '{template_type}' not found. Available: {', '.join(templates.keys())}"

def calculate_task_priority_func(impact: int, urgency: int, effort: int) -> str:
    """
    Calculate task priority score using Eisenhower Matrix principles.
    
    Args:
        impact: Impact score (1-10)
        urgency: Urgency score (1-10)
        effort: Effort required (1-10, where 10 is highest effort)
    """
    # Validate inputs
    for score, name in [(impact, "Impact"), (urgency, "Urgency"), (effort, "Effort")]:
        if not 1 <= score <= 10:
            return f"Error: {name} must be between 1 and 10"
    
    # Calculate priority score (higher impact and urgency, lower effort = higher priority)
    priority_score = (impact * 0.4 + urgency * 0.4) * (11 - effort) / 10
    
    # Determine category
    if urgency >= 7 and impact >= 7:
        category = "Critical - Do First"
    elif urgency < 7 and impact >= 7:
        category = "Important - Schedule"
    elif urgency >= 7 and impact < 7:
        category = "Urgent - Delegate if possible"
    else:
        category = "Low Priority - Consider dropping"
    
    # Get recommendation
    if priority_score >= 7:
        recommendation = "High priority - Address immediately"
    elif priority_score >= 5:
        recommendation = "Medium priority - Schedule within the week"
    elif priority_score >= 3:
        recommendation = "Low priority - Address when time permits"
    else:
        recommendation = "Very low priority - Consider if necessary"
    
    return f"""
    Task Priority Analysis:
    - Impact: {impact}/10
    - Urgency: {urgency}/10
    - Effort: {effort}/10
    - Priority Score: {priority_score:.1f}/10
    - Category: {category}
    - Recommendation: {recommendation}
    """

# Create tools
generate_email_template = tool("Email Template Generator")(generate_email_template_func)
calculate_task_priority = tool("Task Priority Scorer")(calculate_task_priority_func)

# Test functions
print("Testing email template:")
print(generate_email_template_func("welcome", "John Doe"))
print("\nTesting task priority:")
print(calculate_task_priority_func(8, 9, 3))

## 8. Complete Example: Multi-Agent System with Custom Tools

Let's create a complete example showing how different agents use custom tools to solve a business problem.

In [ ]:
# Create specialized agents with custom tools

# Financial Analyst Agent
financial_analyst = Agent(
    role="Financial Analyst",
    goal="Analyze financial metrics and provide investment insights",
    backstory="""
    You are a senior financial analyst with expertise in investment analysis,
    ROI calculations, and financial modeling. You use various financial tools
    to provide data-driven recommendations.
    """,
    tools=[calculate_roi, calculate_clv, calculate_break_even, 
           get_stock_price, convert_currency],
    llm=llm,
    verbose=True
)

# Operations Manager Agent
operations_manager = Agent(
    role="Operations Manager",
    goal="Optimize business operations and workflow efficiency",
    backstory="""
    You are an experienced operations manager who excels at streamlining
    processes, managing resources, and improving operational efficiency
    using various automation and optimization tools.
    """,
    tools=[calculate_task_priority, generate_email_template],
    llm=llm,
    verbose=True
)

# Data Scientist Agent
data_scientist = Agent(
    role="Data Scientist",
    goal="Extract insights from data and create visualizations",
    backstory="""
    You are a skilled data scientist who specializes in data analysis,
    pattern recognition, and creating compelling visualizations to
    communicate insights effectively.
    """,
    tools=[analyze_csv_data, create_data_visualization, 
           transform_json_data, process_text_file],
    llm=llm,
    verbose=True
)

In [ ]:
# Define tasks for a business analysis project

financial_analysis_task = Task(
    description="""
    Analyze the financial viability of expanding into a new market.
    Consider:
    1. Calculate ROI for an initial investment of $500,000 with projected 
       returns of $750,000 after 2 years
    2. Determine break-even point with fixed costs of $100,000, 
       variable cost of $50/unit, and selling price of $120/unit
    3. Check current stock prices for AAPL and MSFT as market indicators
    4. Convert the investment amount to EUR and GBP for international partners
    
    Provide a comprehensive financial assessment.
    """,
    expected_output="Detailed financial analysis with calculations and recommendations",
    agent=financial_analyst
)

operational_planning_task = Task(
    description="""
    Plan the operational aspects of the market expansion:
    1. Prioritize key tasks:
       - Market research (Impact: 9, Urgency: 8, Effort: 6)
       - Team hiring (Impact: 8, Urgency: 7, Effort: 8)
       - Infrastructure setup (Impact: 7, Urgency: 6, Effort: 9)
    2. Generate a welcome email template for new team members
    
    Create an operational roadmap.
    """,
    expected_output="Operational plan with prioritized tasks and communication templates",
    agent=operations_manager
)

data_insights_task = Task(
    description="""
    Analyze market data and create visualizations:
    1. Process the provided market data (create sample data for revenue projections)
    2. Create a visualization showing projected revenue growth
    3. Transform data into different formats for various stakeholders
    
    Deliver data-driven insights for decision making.
    """,
    expected_output="Data analysis report with visualizations and insights",
    agent=data_scientist
)

# Create a crew for the business expansion analysis
business_expansion_crew = Crew(
    agents=[financial_analyst, operations_manager, data_scientist],
    tasks=[financial_analysis_task, operational_planning_task, data_insights_task],
    process=Process.sequential,
    verbose=True
)

In [ ]:
# Execute the business expansion analysis
print("Starting Business Expansion Analysis...\n")
print("=" * 50)

result = business_expansion_crew.kickoff()

print("\n" + "=" * 50)
print("BUSINESS EXPANSION ANALYSIS COMPLETE")
print("=" * 50)
print(result)

## 9. Advanced Tool Patterns

Here are some advanced patterns for creating more sophisticated custom tools.

In [ ]:
# Tool with error handling and validation
def advanced_calculator_func(expression: str, variables: dict = None) -> str:
    """
    Evaluate mathematical expressions with variables.
    
    Args:
        expression: Mathematical expression to evaluate
        variables: Dictionary of variable values
    """
    try:
        import math
        # Security: Only allow safe operations
        allowed_names = {
            k: v for k, v in math.__dict__.items() if not k.startswith("__")
        }
        
        if variables:
            allowed_names.update(variables)
        
        # Evaluate expression
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        
        return f"""
        Expression: {expression}
        Variables: {variables if variables else 'None'}
        Result: {result}
        Type: {type(result).__name__}
        """
    except Exception as e:
        return f"Error evaluating expression: {str(e)}"

# Tool with caching capability
_cache = {}

def fetch_with_cache_func(key: str, fetch_new: bool = False) -> str:
    """
    Fetch data with caching to improve performance.
    
    Args:
        key: Data key to fetch
        fetch_new: Force fetch new data
    """
    global _cache
    
    if not fetch_new and key in _cache:
        return f"Cached data for '{key}': {_cache[key]['data']}\n(Cached at: {_cache[key]['timestamp']})"
    
    # Simulate fetching new data
    import time
    data = f"Fresh data for {key} at {datetime.now()}"
    
    _cache[key] = {
        'data': data,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    return f"Fetched new data for '{key}': {data}"

# Create tools
advanced_calculator = tool("Advanced Calculator")(advanced_calculator_func)
fetch_with_cache = tool("Cached Data Fetcher")(fetch_with_cache_func)

# Test the functions
print("Testing advanced calculator:")
print(advanced_calculator_func("sqrt(x) + y**2", {"x": 16, "y": 3}))

print("\nTesting cache:")
print(fetch_with_cache_func("user_data"))
print(fetch_with_cache_func("user_data"))  # Should return cached data

## 10. Best Practices for Custom Tools

### 1. **Clear Documentation**
- Always include detailed docstrings
- Specify parameter types and expected formats
- Provide examples in the description

### 2. **Error Handling**
- Wrap operations in try-except blocks
- Return meaningful error messages
- Validate inputs before processing

### 3. **Return Format**
- Return structured, readable text
- Include relevant context in responses
- Format numbers and data appropriately

### 4. **Performance**
- Implement caching for expensive operations
- Use batch processing when possible
- Avoid blocking operations

### 5. **Security**
- Sanitize inputs, especially for file operations
- Avoid executing arbitrary code
- Limit access to sensitive resources

### 6. **Modularity**
- Keep tools focused on single responsibilities
- Create composable tools that work well together
- Avoid overly complex tools

### 7. **Testing**
- Test tools independently before integration
- Handle edge cases gracefully
- Provide sample data for testing

## Key Takeaway: Working with Tools

When creating custom tools in CrewAI:

1. **The `@tool` decorator transforms functions into tool objects** that agents can use, but these objects are not directly callable

2. **Best Practice Pattern:**
   ```python
   # Step 1: Define your function
   def my_function(param: str) -> str:
       return f"Result: {param}"
   
   # Step 2: Create the tool
   my_tool = tool("My Tool Name")(my_function)
   
   # Step 3: Test using the original function
   print(my_function("test"))  # This works!
   
   # Step 4: Use the tool with agents
   agent = Agent(tools=[my_tool], ...)
   ```

3. **This pattern gives you:**
   - Easy testing of your logic
   - Clear separation between function logic and tool creation
   - Flexibility to reuse functions in different contexts

## Summary

This notebook demonstrated various applications of custom tools in CrewAI:

1. **Data Analysis** - Tools for processing and analyzing data
2. **API Integration** - Connecting to external services
3. **File Processing** - Handling various file formats
4. **Business Logic** - Implementing domain-specific calculations
5. **Automation** - Streamlining workflows and tasks

Custom tools extend agent capabilities, allowing them to:
- Perform specialized calculations
- Integrate with external systems
- Process complex data
- Automate repetitive tasks
- Implement business logic

By using function-based tools with the `tool` decorator, you can quickly create powerful extensions for your AI agents without the complexity of class-based implementations.